# Downloder

In this Jupyter notebook we filter the image data, select the models and images and download thi data. 

In [ ]:
'''
source: https://medium.com/@adrianmrit/creating-simple-image-datasets-with-flickr-api-2f19c164d82f
'''
from flickrapi import FlickrAPI
import os
import time
from progress.bar import Bar
import requests
import os
import sys
import time
from PIL import Image
from PIL.ExifTags import TAGS
import pandas as pd
import numpy as np
import pickle

KEY = 'YourFlickrAPIKey'
SECRET = 'Mellon'


In [ ]:
# Create dataframe from our main files
# Link: https://faui1-gitlab.cs.fau.de/mullanptr/flickr_data
# files are merged on 'imgID'
df3 = pd.merge(
    pd.read_csv('initial_data.csv'),
    pd.read_csv('parsed_data.csv'),
    on='imgID',
    how='outer')
df3.head()

In [ ]:
# Get most common makes of images 
# set number of best models
num_best = 9
common_make = list(df3["Make_normalized"].value_counts()[:num_best].index)
print(common_make)

# Filter the dataset so we have only common makes makes 
df_filtered = df3.loc[df3['Make_normalized'].isin(common_make)]
df_filtered.info()


In [ ]:
uniques = df_filtered['unique_id'].unique()
serija = df_filtered["unique_id"].value_counts().describe()
#df_cut = 
#
counts = df_filtered['unique_id'].value_counts().rename('unique_counts')
print(counts)
df_filtered1 = df_filtered.merge(counts.to_frame(),
                                left_on='unique_id',
                                right_index=True)

In [ ]:
# We wil check for the occurences of each unique combination, 
# for start lets just get unique combinations
makes = pd.unique(df_filtered[["Make_normalized"]].values.ravel())
df_filtered["unique_id"] = -1
c = 0

for make in makes: 
    sub_df = df_filtered.loc[df_filtered["Make_normalized"] == make]
    models = pd.unique(sub_df[["Model_normalized"]].values.ravel())
    print(make, sub_df.shape, len(models))
    for model in models: 
        #print("     -", model)
        df_filtered.loc[(df_filtered['Make_normalized'] == make) &
                        (df_filtered['Model_normalized'] == model), 
                        'unique_id'] = c
        c+=1        
    
    

In [ ]:
# check the uniques
uniques = df_filtered['unique_id'].unique()
# I think this is useless
serija = df_filtered["unique_id"].value_counts().describe()

# Count the number of unique occurences
counts = df_filtered['unique_id'].value_counts().rename('unique_counts')
print(counts)
# Megre with df 
df_filtered1 = df_filtered.merge(counts.to_frame(),
                                left_on='unique_id',
                                right_index=True)

In [ ]:
# creating reserve df to have backup if anything is deleted
df_reserve = df_filtered
df_filtered = df_filtered1

In [ ]:
# Now lets select the combinations of camera makes and models that has more than 200 images 
df_filtered = df_filtered[df_filtered.unique_counts > 200]
#cdf.info()
#cdf.head()

In [ ]:
import random
# Generate empty dataframe that we will use for download 
df_download = df_filtered.loc[df_filtered['Make_normalized']=='Generate_empty']

#sample random n_sample samples. 

# ** NOTE We have set 9 common makes 
# If number of downloaded images will be then 9* n_sample* n_models
# 
n_sample = 200
n_models = 20
i = 0

for make in common_make:  
    print(make)
    sub_df = df_filtered.loc[df_filtered["Make_normalized"] == make]
    models = pd.unique(sub_df[["Model_normalized"]].values.ravel())
    
    #print(type(models))
    #print(rmodels)
    
    j = 0
    while j < n_models:
        # Select random model 
        model = np.random.choice(models, 1)[0]
        #print(model)
        dfh = df_filtered[(df_filtered['Make_normalized']==make)&
                          (df_filtered['Model_normalized'] == model)]
        #print(dfh.shape)
        
        # This is a check that should never be True. 
        if dfh.shape[0] < n_sample: 
            continue
        j+=1
        
        # Sample n images 
        df_subset = df_filtered[(df_filtered['Make_normalized']==make)&
                                (df_filtered['Model_normalized'] == model)].sample(n=n_sample)
        df_subset['type'] = i
        df_download = pd.concat([df_download, df_subset])
print(df_download.info())
    
# We get the dataframe with url's we will download
#df_download.head(20),,


In [ ]:
# Download images and save data

# 1. download image, save EXIF
data_dict = dict()
count = 0
for index, row in df_download.iterrows():
    if (count+1)%100 == 0: 
        print(count+1)
    url = row['url_o']
    response=requests.get(url,stream=True) 
    
    # Download image, get EXIF DATA
    try: 
        image = Image.open(requests.get(url, stream=True).raw)
        exifdata = image.getexif()
        
        #Save EXIF data in dict <-- this dictionary is the goal of Downloader 
        data_dict[url] = exifdata
       
            
    except Exception as e: 
        #print("Problem - " , url, ": ",  e)
        try: 
            df_download = df_download.drop(index)
        except: 
            pass
    count +=1

# 2. Save 'data_dict' with exif info and 'download_df' with normalized data info
with open('filtered9b.pickle', 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('download_meta9b.pickle', 'wb') as handle:
    pickle.dump(df_download, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("DONE!")

In [ ]:
df_download.info()